                                   Roujia Wen


# PuzzleNode

In [1]:
import numpy as np
from Queue import PriorityQueue, Queue

class PuzzleNode(object):
    """Represents a node in the search. It saves the state, parent,
    path cost and heuristic value and other relevant information. It
    is able to generate a child node given an action.

    Parameters
    ----------
    state : tuple of tuple
        State of the puzzle in this node.
    empty : list of int
        Location of the empty cell in the puzzle, following the format: 
        [axis1_index, axis2_index]. Purpose of recording location is so
        that it can be modified and passed on to the next node, therefore
        the empty cell doesn't need to be looked for every time.
    parent: `PuzzleNode` object
        Parent of this node in the search tree. This is used for retriev-
        ing steps of the solution.
    path_cost: int
        Path cost from the initial node to this node.

    Attributes
    ----------
    hashed : int
        Hash of the state tuple; used for explored list
    n : int
        Dimension of the puzzle state.
    h : int
        Estimated cost of the cheapest path from this node to the goal.
    f : int
        Estimated cost of the cheapest path from the initial node to the
        goal through this node.
    state, empty, parent, path_cost: 
        Same as in the ``Parameters`` section.
        
    Methods
    -------
    child(action)
        Generates a child node given an action.
    """
    def __init__(self, state, empty, parent, path_cost):
        self.state = state
        self.hashed = hash(state)
        self.empty = empty
        self.n = len(state)
        self.parent = parent
        self.path_cost = path_cost
        self.h = CURRENT_HEURISTIC(state)
        self.f = self.path_cost + self.h
        
    def __lt__(self, other):
        """This defines priority of nodes according to their `f` value,
        consistent with A-star search.
        """
        return self.f < other.f
    
    def __str__(self):
        """String representation of the state.
        e.g.
        +---+---+---+
        | 0 | 1 | 2 |
        +---+---+---+
        | 3 | 4 | 5 |
        +---+---+---+
        | 6 | 7 | 8 |
        +---+---+---+
        """
        max_len = max(len(str(self.n**2-1)), 3)
        divider = ("-"*max_len).join(["+" for _ in range(self.n+1)])
        s = divider+"\n"
        for row in self.state:
            string_list = [('{:^%dd}'%(max_len)).format(_) for _ in row]
            s += "|"+"|".join(string_list)+"|\n"+divider+"\n"
        return s
    
    def child(self, action):
        """Generate and return a child of this node given an action.
        
        Parameters
        ----------
        action : {'up', 'down', 'left', 'right'}
            Action to be executed on this node.

        Returns
        -------
        `PuzzleNode` object or `None`
            Child node generated from this node given the action. Returns
            `None` if empty cell is on the boundary and action cannot be
            taken.
        """
        
        # Copy the state tuple into a 2D list
        arr = [list(_) for _ in self.state]
    
        # Row (axis1) and column (axis2) index of the empty cell
        r, c = self.empty[0], self.empty[1]

        # Determine the location of the new empty cell
        if action == "up":
            if r == self.n-1: return None
            new_empty = [r+1, c]
        elif action == "down":
            if r == 0: return None
            new_empty = [r-1, c]
        elif action == "left":
            if c == self.n-1: return None
            new_empty = [r, c+1]
        elif action == "right":
            if c == 0: return None
            new_empty = [r, c-1]
        else:
            print "Error: not a valid direction."
        
        # Swap the empty cell into its destination
        arr[r][c], arr[new_empty[0]][new_empty[1]] =\
                arr[new_empty[0]][new_empty[1]], arr[r][c]
        
        # Change the type from list to tuple to insert into child node
        arr = tuple([tuple(_) for _ in arr])
        
        return PuzzleNode(arr, new_empty, self, self.path_cost+1)


# Functions for Input and Output 

In [2]:
def check_state_format(state, n):
    """Check whether a given state is in the correct format.
    
    Parameters
    ----------
    state : list of lists
        Represents a state of the puzzle board.
    n : int
        Dimension of the board is n x n.

    Returns
    -------
    bool
        True if format is correct; False otherwise
    """
    
    if not isinstance(state, list):
        print "Error: initial state is not a list"
        return False

    state = np.array(state)
    s = state.shape
    if (len(s)!=2) or (s[0]!=n) or (s[1]!=n):
        print "Error: initial state is not of the shape\
({},{})".format(n,n)
        return False
        
    if not np.issubdtype(state.dtype, np.integer):
        print "Error: initial state entries contain non-integer"
        return False

    if np.any(state>=n*n) or np.any(state<0):
        print "Error: initial state entries contain out-of-range numbers"
        return False
    
    count = np.zeros(n*n)
    count[state] += 1
    if np.count_nonzero(count-1) != 0:
        print "Error: entries do not contain each number exactly once"
        return False
    
    return True

def check_solvable(state):
    """Check whether a given state is solvable using invariants.
    Principles explained in http://www.cs.princeton.edu/courses
    /archive/fall12/cos226/assignments/8puzzle.html
    
    Parameters
    ----------
    state : list of lists
        Represents a state of the puzzle board.

    Returns
    -------
    bool
        True if solvable; False otherwise
    """
    def count_inversions(state):
        lst = np.array(state).flatten()
        count = 0
        for i in range(len(lst)-1):
            count += np.count_nonzero(lst[i] > lst[i+1:])
        zero_index = np.where(lst==0)[0][0]
        count -= zero_index #subtract the inversions of blank
        return count
    
    def count_row_of_blank(state):
        temp = [_ for _ in state if 0 in _][0]
        return state.index(temp)
    
    def is_odd(v):
        if v%2 == 1: return True
        return False
    
    n = len(state)
    inversions = count_inversions(state)
    row_of_blank = count_row_of_blank(state)
    
    # If size is odd, number of inversions need to be odd
    is_case1 = is_odd(n) and is_odd(inversions)
    # If size is even, number of inversions+row number of blank
    # has to be odd
    is_case2 = (not is_odd(n)) and is_odd(inversions+row_of_blank)
    if is_case1 or is_case2: return False
    # Otherwise unsolvable
    return True

def recursive_print(node):
    """ Recursively prints out the solution and step number.
    """
    if node.parent is None:
        print node.path_cost
        print node
        return
    recursive_print(node.parent)
    print node.path_cost
    print node


# A\* Search

In [3]:
# Set the level of detailedness for verbose printing
VERBOSE_LEVEL = 0

def vprint(threshold, title, message=None, exact=False):
    """For printing out extra information in each search step.
    """
    if ((VERBOSE_LEVEL > threshold) and (not exact)) or\
        ((VERBOSE_LEVEL == threshold) and exact):
        print title
        if message is None:
            return
        if isinstance(message, list) or isinstance(message, tuple):
            print np.array(message) # Call numpy for pretty printing
        elif isinstance(message, dict):
            print "\n".join([str(np.array(_.state)) 
                             for _ in message.values()])
        elif isinstance(message, PriorityQueue):
            print "\n".join([str(np.array(_.state)) 
                             for _ in message.queue])
        else:
            print message

def solvePuzzle(n, initial_state, heuristic, print_=False):
    """Solve a puzzle game given an initial state and a heuristic using
    A-star search.

    Parameters
    ----------
    n : int
        Dimension of the puzzle board.
    initial_state : list of lists
        The initial state of the puzzle board.
    heuristic : pointer to a function
        Specifies the heuristic function which will be used.
    print_ : bool, default False
        Specifies whether to print out the solution if it exists.

    Returns
    -------
    int
        Number of steps required to reach the goal.
    int
        Maximum size of the frontier during the search.
    int
        Error code. 0 if no error; -1 if input format is incorrect;
        -2 if state is unsolvable; -3 if pattern database has not been
        built.
    """
    
    if heuristic == h_pattern_database:
        if (not 'PDB' in globals()) or (n not in PDB):
            print "Error: Pattern database of n={} has not been built. \
Please run `builtPatternDatabase` in the next cell.".format(n)
            return 0, 0, -3
    
    # Set heuristic
    global CURRENT_HEURISTIC
    CURRENT_HEURISTIC = heuristic
    
    # Check if input format is correct
    if not check_state_format(initial_state, n): return 0, 0, -1
    
    # Check if state is solvable
    if not check_solvable(initial_state): return 0, 0, -2
    
    # Change state data type from list to tuple for easy look up
    initial_state = tuple([tuple(_) for _ in initial_state])
    goal_state = tuple([tuple([i*n+j for j in range(n)])
                        for i in range(n)])
    
    # Find location for the empty cell marked with 0
    temp = [_ for _ in initial_state if 0 in _][0]
    initial_empty_loc = [initial_state.index(temp), temp.index(0)]

    # Initialize the first node; parent=None and path_cost=0
    node = PuzzleNode(initial_state, initial_empty_loc, None, 0)
    
    # Initialize frontier, explored list, and others
    frontier = PriorityQueue()
    frontier.put(node)
    explored = {}
    explored[node.hashed] = node
    search_step = 0
    max_frontier = 0
    
    while True:
        search_step += 1
        vprint(0, "##########STEP={}############".format(search_step))
        
        # If frontier is empty, search has failed to reach a solution
        if frontier.empty(): return 0, 0, -2
        
        # Track maximum frontier size
        max_frontier = max(max_frontier, frontier.qsize())
        
        # Choose the lowest-cost node from frontier (priority queue)
        node = frontier.get()
        vprint(0, "POP", node.state)
        vprint(0, "pathcost = {}".format(node.path_cost))

        # If the same state has been explored, skip the search step
        if (node.hashed in explored) and (explored[node.hashed] < node):
            vprint(1, "ALREADY EXPLORED; SKIP")
            continue
        
        # Check if solution has been reached
        if node.state == goal_state:
            if print_:
                recursive_print(node)
                print "Num of Moves = {}; Max Frontier Size = {}".format(
                    node.path_cost, max_frontier)
            return node.path_cost, max_frontier, 0
        
            
        # For each possible action, generate a child node
        for each_action in ["up", "down", "left", "right"]:
            vprint(1, "-------  {}  -------".format(each_action))
            child = node.child(each_action)
            # If search did not hit the boundary
            if child is not None:
                not_explored = child.hashed not in explored
                # If the state has not been explored, or better cost been
                # found, add to the frontier, and update explored list.
                if not_explored or (child < explored[child.hashed]):
                    vprint(0, "ADD NODE", child.state)
                    vprint(1, "unexplored = {}".format(not_explored))
                    vprint(0, "pathcost = {}".format(child.path_cost))
                    explored[child.hashed] = child
                    frontier.put(child)
        vprint(1, "-----------------------")   
        vprint(1, "<<<EXPLORED LIST>>>", explored)            
        vprint(1, "<<<FRONTIER>>>", frontier)

# Heuristics

In [4]:
class memoize:
    """Wrapper for caching heuristic function outputs.
    """
    def __init__(self, f):
        self.f = f
        self.cache = {}
    def __call__(self, arg):
        if not isinstance(arg, tuple):
            arg = tuple([tuple(_) for _ in arg])
        if not arg in self.cache:
            self.cache[arg] = self.f(arg)
        return self.cache[arg]

@memoize
def h_misplaced(state):
    """Return estimated cheapest path cost from given state to the goal
    state using the number of misplaced cells.
    """
    count = 0
    for i in range(len(state)):
        for j in range(len(state)):
            if (state[i][j] != i*len(state)+j) and (state[i][j] != 0):
                count += 1
    return count

@memoize
def h_manhattan(state):
    """Return estimated cheapest path cost from given state to the goal
    state using total manhattan distance.
    """
    count = 0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0:
                p = state[i][j]/len(state)
                q = state[i][j]%len(state)
                count += abs(p-i)+abs(q-j)
    return count

@memoize
def h_pattern_database(state):
    """Return estimated cheapest path cost from given state to the goal
    state using pattern database, plus manhattan distances for un-
    specified cells.
    
    Cells in the specified pattern are marked with brackets below.
    
    When n=3:
        (0)(1)(2)
        (3) 9  9
        (6) 9  9
        Also known as the fringe pattern.
        
    When n=4:
        (0) (1) (2) (3)
        (4)  16  16  16
         16  16  16  16
         16  16  16  16
        Using top row plus an extra cell.
        (Chose this pattern instead of the fringe due to constraint
        in computing time)
        
    When n>4:
        Using the top row as the pattern.
        (It will take very long to run. It's outside the scope of the
        design of this code)
    """
    n = len(state)
    
    if n<3:
        return h_manhattan(state)
    elif n==3: 
        specified = set([0, 1, 2, 3, 6])
    elif n==4:
        specified = set([0, 1, 2, 3, 4])
    else:
        specified = set(range(n))
    
    count = 0 #manhattan distance count
    pattern = [list(_) for _ in state] # pattern used for look-up in PDB
    for i in range(n):
        for j in range(n):
            if state[i][j] not in specified:
                # Only count manhattan distance for unspecified cells
                p = state[i][j]/n
                q = state[i][j]%n
                count += abs(p-i)+abs(q-j)
                pattern[i][j] = n*n #mask unspecified cells
    pattern = tuple([tuple(_) for _ in pattern])
    return count + PDB[n][hash(pattern)] #manhanttan + PDB

# A list of references to different heuristic functions
heuristics = [h_misplaced, h_manhattan, h_pattern_database] 

## Building Pattern Database

In [5]:
from math import factorial
class PatternNode(object):
    """ Adapted from PuzzleNode (see Section 1). Removed unnecessary 
    methods. Added ``actual_cost``.

    Additional Parameter
    --------------------
    actual_cost: int
        Path cost from the initial node to this node, excluding moves
        associated with unspecified cells.

    """
    def __init__(self, state, empty, parent, actual_cost):
        self.state = state
        self.hashed = hash(state)
        self.empty = empty
        self.n = len(state)
        self.parent = parent
        self.actual_cost = actual_cost

    def child(self, action):        
        # Copy the state tuple into a 2D list
        arr = [list(_) for _ in self.state]
    
        # Row (axis1) and column (axis2) index of the empty cell
        r, c = self.empty[0], self.empty[1]

        # Determine the location of the new empty cell
        if action == "up":
            if r == self.n-1: return None
            new_empty = [r+1, c]
        elif action == "down":
            if r == 0: return None
            new_empty = [r-1, c]
        elif action == "left":
            if c == self.n-1: return None
            new_empty = [r, c+1]
        elif action == "right":
            if c == 0: return None
            new_empty = [r, c-1]
        else:
            print "Error: not a valid direction."
        
        # Swap the empty cell into its destination
        arr[r][c], arr[new_empty[0]][new_empty[1]] =\
                arr[new_empty[0]][new_empty[1]], arr[r][c]
        
        # Change the type from list to tuple to insert into child node
        arr = tuple([tuple(_) for _ in arr])
        
        # If the cell being slided is unspecified, ignore in actual cost
        if arr[r][c] == self.n**2:
            return PatternNode(arr, new_empty, self, self.actual_cost)
        else:
            return PatternNode(arr, new_empty, self, self.actual_cost+1)

def buildPatternDatabase(n):
    """Adapted from solvePuzzle (see Section 3), main changes are:
        1) This is a backward search with initial state as the target
           pattern, and the goal is to enumerate all permutations.
        2) Therefore, no specific goal test is performed, search stops
           simply when the frontier is empty.
        3) State representation:
             (0)(1)(2)
             (3) 9  9
             (6) 9  9
           - Specified cells (bracketed) remain the same while un-
             specified cells are all represented by n*n.
           - See docstrings for `h_pattern_database` for more on spe-
             cified patterns for when n=3, n=4 vs. n>4.
        4) Since this is a bredth-first search:
           - No heuristic function is used.
           - Frontier uses Queue rather than PriorityQueue.

    Parameters
    ----------
    n : int
        Dimension of the puzzle board.

    Returns
    -------
    dict
        Pattern database. Keys are hashed 2D tuple representing states
        and values are the minimum cost from the state to solved state.
    """

    
    # Set initial state for backward search(which is the target pattern)
    if n < 3:
        print "Error: pattern database not necessary for n < 3"
    elif n == 3:
        # Use the fringe pattern
        initial_state = [[0, 1, 2],[3, 9, 9],[6, 9, 9]]
    elif n == 4:
        # Use top row + one extra cell pattern
        initial_state = [[0, 1, 2, 3],
                         [4,16,16,16],
                         [16,16,16,16],
                         [16,16,16,16]]
    else:
        # Otherwise, just use the top row pattern
        initial_state = [[i*n+j if (i==0) else n*n for j 
                          in range(n)] for i in range(n)]
    
    # Number of possible permutations (only considering specified cells)
    temp = np.ravel(initial_state) #flatten
    goal_length = n*n
    for i in range(n*n-(len(set(temp))-1)+1,n*n): goal_length *= i
    
    # Convert type to tuple
    initial_state = tuple([tuple(_) for _ in initial_state])
    
    # Location for the empty cell marked with 0
    initial_empty_loc = [0, 0]

    # Initialize the first node; parent=None and actual_cost=0
    node = PatternNode(initial_state, initial_empty_loc, None, 0)
    
    # Initialize frontier, explored list
    frontier = Queue()
    frontier.put(node)
    explored = {}
    explored[node.hashed] = node.actual_cost
    
    # Stops when fontier is empty
    while not frontier.empty():
        # Choose the oldest node from frontier
        node = frontier.get()
        # For each possible action, generate a child node
        for each_action in ["up", "down", "left", "right"]:
            child = node.child(each_action)
            # If search did not hit the boundary 
            if child is not None:
                # If state is not explored or better cost
                if ((child.hashed not in explored) or 
                        (child.actual_cost < explored[child.hashed])):
                    explored[child.hashed] = child.actual_cost
                    frontier.put(child)
    
    return explored #the explored dictionary is the pattern database

"""Add pattern database of n=3 and n=4; could add other n values but it 
will take a long time.
"""
PDB = {}
PDB[3]= buildPatternDatabase(3) # run time ~ 1.6s
PDB[4]= buildPatternDatabase(4) # run time ~ 17s
print "Success!"

Success!


# Problems and Tests

In [6]:
# n=3
p3_1 = [[5,7,6],[2,4,3],[8,1,0]]
p3_2 = [[7,0,8],[4,6,1],[5,3,2]]
p3_3 = [[2,3,7],[1,8,0],[6,5,4]]
p3_4 = [[8,6,7],[2,5,4],[3,0,1]]

# n=4
p4_1 = [[7,2,4,9],[5,10,0,6],[8,11,3,1],[12,13,14,15]]
p4_2 = [[9,14,5,0], [1,4,2,3], [8,11,7,6], [12,10,15,13]]
p4_3 = [[1,2,3,4],[10,8,12,6],[14,5,0,15],[9,13,7,11]]
p4_4 = [[13,9,7,15],[3,6,8,4],[11,10,2,12],[5,14,1,0]]

# Unsolvable problems
u3_1 = [[8,2,3],[1,6,7],[0,4,5]]
u3_2 = [[7,4,2],[5,0,6],[8,3,1]]
u4_1 = [[1,2,0,4],[7,10,6,8],[9,14,12,11],[13,5,15,3]]
u4_2 = [[1,12,10,2],[6,0,7,8],[9,14,5,11],[13,15,4,3]]

# Erroneous input
e1 = [[1,2,3],[1,2,3],[1,2,3]]
e2 = [[1,2,3],[4,5,6],[7,8,9]]
e3 = [[1,2,3],[4,5,6],[7,8]]
e4 = [[1,2,3],[4,5,6],[7,8,"o"]]


In [7]:
# n=3
for prob in [p3_1, p3_2, p3_3, p3_4]:
    for i, h in enumerate(["misplaced", "manhattan","pattern DB"]): 
        print h, ":", solvePuzzle(3, prob, heuristics[i])
    print "--------------------"

misplaced : (28, 21008, 0)
manhattan : (28, 922, 0)
pattern DB : (28, 494, 0)
--------------------
misplaced : (25, 12911, 0)
manhattan : (25, 859, 0)
pattern DB : (25, 349, 0)
--------------------
misplaced : (17, 559, 0)
manhattan : (17, 67, 0)
pattern DB : (17, 54, 0)
--------------------
misplaced : (27, 20068, 0)
manhattan : (27, 1656, 0)
pattern DB : (27, 603, 0)
--------------------


In [8]:
# n=4 
for prob in [p4_1, p4_2, p4_3, p4_4]:
    # Deleted "misplaced" because it will take too long
    for i, h in enumerate(["manhattan","pattern DB"]): 
        print h, ":", solvePuzzle(4, prob, heuristics[i+1])
    print "--------------------"


manhattan : (33, 15417, 0)
pattern DB : (33, 5059, 0)
--------------------
manhattan : (33, 7422, 0)
pattern DB : (33, 5527, 0)
--------------------
manhattan : (38, 69297, 0)
pattern DB : (38, 53831, 0)
--------------------
manhattan : (54, 623112, 0)
pattern DB : (54, 188869, 0)
--------------------


In [9]:
# Unsolvable
for prob in [u3_1, u3_2, u4_1, u4_2]:
    print solvePuzzle(len(prob), prob, heuristics[i])
    print "--------------------"
    
# Erroneous input
for prob in [e1, e2, e3, e4]:
    print solvePuzzle(len(prob), prob, heuristics[i])
    print "--------------------"
    

(0, 0, -2)
--------------------
(0, 0, -2)
--------------------
(0, 0, -2)
--------------------
(0, 0, -2)
--------------------
Error: entries do not contain each number exactly once
(0, 0, -1)
--------------------
Error: initial state entries contain out-of-range numbers
(0, 0, -1)
--------------------
Error: initial state is not of the shape(3,3)
(0, 0, -1)
--------------------
Error: initial state entries contain non-integer
(0, 0, -1)
--------------------


In [10]:
# Print solution
solvePuzzle(3, [[1,4,2],[3,7,5],[6,8,0]], heuristics[2], print_=True)

0
+---+---+---+
| 1 | 4 | 2 |
+---+---+---+
| 3 | 7 | 5 |
+---+---+---+
| 6 | 8 | 0 |
+---+---+---+

1
+---+---+---+
| 1 | 4 | 2 |
+---+---+---+
| 3 | 7 | 5 |
+---+---+---+
| 6 | 0 | 8 |
+---+---+---+

2
+---+---+---+
| 1 | 4 | 2 |
+---+---+---+
| 3 | 0 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+

3
+---+---+---+
| 1 | 0 | 2 |
+---+---+---+
| 3 | 4 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+

4
+---+---+---+
| 0 | 1 | 2 |
+---+---+---+
| 3 | 4 | 5 |
+---+---+---+
| 6 | 7 | 8 |
+---+---+---+

Num of Moves = 4; Max Frontier Size = 6


(4, 6, 0)

# Discussions

#### Solving the problems in the step 4 in assignment instructions, we have:  
[[5,7,6],[2,4,3],[8,1,0]]

* misplaced : (28, 21008, 0)
* manhattan : (28, 922, 0)
* pattern DB : (28, 494, 0)

\-------------------------

[[7,0,8],[4,6,1],[5,3,2]]

* misplaced : (25, 12911, 0)
* manhattan : (25, 859, 0)
* pattern DB : (25, 349, 0)

\-------------------------

[[2,3,7],[1,8,0],[6,5,4]]

* misplaced : (17, 559, 0)
* manhattan : (17, 67, 0)
* pattern DB : (17, 54, 0)

We can see that different heuristics have the same number of moves,
this is because they are are all admissible heuristics, and so A\*
search is able to find optimal solutions.

However, the max frontier size differs quite a lot. We can see that
$MISPLACED$ is one or two orders of magnitude worse than $MANHATTAN$, which is 
again about half an order of magnitude worse than $PATTERNDB$ (It's more 
significant when n=4 than when n=3).

#### Why does pattern database heuristic dominate the other two heuristics?  
Definition of dominance: if for any node n and heuristics $h_1$ and $h_2$, $h_2(n) \geq h_1(n)$, we say that $h_2$ dominates $h_1$ $^{[1]}$.

1. $MANHATTAN$ dominates $MISPLACED$:
    We can show that for any state i, $MANHATTAN(i) \geq MISPLACED(i)$. This is because for each tile t, if it's in the correct position, than both $MANHATTAN$ and $MISPLACED$ would be zero therefore equal; otherwise $MANHATTAN$ will be at least 1 while $MISPLACED$ is 1. Therefore $MANHATTAN(t) \geq MISPLACED(t)$. When we sum all the tiles up, $MANHATTAN(i) \geq MISPLACED(i)$ still holds.
    
    
2. PATTERNDB dominates $MANHATTAN$:
    For unspecified tiles u, $PATTERNDB(u) = MANHATTAN(u)$. For any specified tile s, $PATTERNDB(s) \geq MANHATTAN(s)$. This is because $PATTERNDB(s)$ is the actual steps that s takes to get to its correct position, while $MANHATTAN(s)$ is the shortest path, which means take it doesn't consider the fact that moving s might interfere with other tiles' position and therefore costing more. Summing up unspecified tiles and specified tiles, we still have $PATTERNDB > MANHATTAN$.
    
Combining (1) and (2), we see that the pattern database heuristic dominates both manhattan distance and number of misplaced cells.






----------

$^{[1]}$Russell, S. & Norvig, P. (1995). A modern approach. *Artificial Intelligence*. Prentice-Hall, Egnlewood Cliffs, 25, 27.
